<a href="https://colab.research.google.com/github/funkierfob3180/Home_Sales/blob/main/Home_Sales_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done


In [3]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType
import time

# Create a SparkSession
spark = SparkSession.builder.appName("TempViews").config("spark.executor.memory","40g").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
#url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_revised = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=',', header=True)
# 2. Print the schema of the DataFrame.
home_sales_revised.printSchema()
# 3. Show the first 5 rows of the DataFrame.
home_sales_revised.show(5)




root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|       

In [47]:
# 2. Create a temporary view of the DataFrame.
#create a temporary view of the DataFrame
home_sales_revised.createOrReplaceTempView("home_sales")



In [88]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""SELECT  ROUND(AVG(price), 2) AS avg_price,date_built,bedrooms FROM home_sales WHERE bedrooms = 4  GROUP BY date_built, bedrooms ORDER BY avg_price DESC""").show()





+---------+----------+--------+
|avg_price|date_built|bedrooms|
+---------+----------+--------+
|307908.86|      2015|       4|
| 302141.9|      2011|       4|
|299999.39|      2013|       4|
|299073.89|      2014|       4|
|298233.42|      2012|       4|
|296800.75|      2010|       4|
|296576.69|      2017|       4|
|296050.24|      2016|       4|
+---------+----------+--------+



In [103]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""
    SELECT 
    EXTRACT(YEAR FROM date_built) AS year,
    CONCAT('$', round(avg(price), 2)) AS avg_price 
    FROM home_sales 
    WHERE bedrooms = 3 and bathrooms = 3 GROUP BY year
    """).show()



+----+----------+
|year| avg_price|
+----+----------+
|2015| $288770.3|
|2013|$295962.27|
|2014|$290852.27|
|2012|$293683.19|
|2016|$290555.07|
|2010|$292859.62|
|2011|$291117.47|
|2017|$292676.79|
+----+----------+



In [127]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""SELECT 
          EXTRACT(YEAR FROM date_built) AS year,
          CONCAT('$', round(avg(price), 2)) AS avg_price 
          FROM home_sales
          where CAST(sqft_living AS INT) >= 2000 and bedrooms = '3'  and bathrooms = '3' and floors = '2' 
          GROUP BY year
          """).show(20)


+----+----------+
|year| avg_price|
+----+----------+
|2015|$297609.97|
|2013|$303676.79|
|2014|$298264.72|
|2012|$307539.97|
|2016| $293965.1|
|2010|$285010.22|
|2011|$276553.81|
|2017|$280317.58|
+----+----------+



In [82]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""SELECT ROUND(AVG(price), 2) AS avg_price, view FROM home_sales WHERE price >= 350000 group by view""").show()



print(time.time() - start_time)

print("--- %s seconds ---" % (time.time() - start_time))


+----------+----+
| avg_price|view|
+----------+----+
| 403005.77|   7|
| 788128.21|  51|
|  404673.3|  15|
| 798684.82|  54|
| 399548.12|  11|
| 397771.65|  29|
| 750537.94|  69|
|  396964.5|  42|
| 1072285.2|  87|
| 752861.18|  73|
| 767036.67|  64|
|  398867.6|   3|
|  397862.0|  30|
| 401419.75|  34|
|  791453.0|  59|
| 398592.71|   8|
| 402124.62|  28|
| 402022.68|  22|
|1056336.74|  85|
| 399586.53|  16|
+----------+----+
only showing top 20 rows

0.8696627616882324
--- 0.8698863983154297 seconds ---


In [108]:
# 7. Cache the the temporary table home_sales.

spark.sql("cache table home_sales")



DataFrame[]

In [109]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [118]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()
spark.sql("""SELECT cast(view as int),CONCAT('$', ROUND(AVG(price), 2)) AS avg_price FROM home_sales WHERE price >= 350000 group by view ORDER BY VIEW""").show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| avg_price|
+----+----------+
|   0|$403848.51|
|   1|$401044.25|
|   2|$397389.25|
|   3| $398867.6|
|   4|$399631.89|
|   5|$401471.82|
|   6|$395655.38|
|   7|$403005.77|
|   8|$398592.71|
|   9|$401393.34|
|  10|$401868.43|
|  11|$399548.12|
|  12|$401501.32|
|  13|$398917.98|
|  14|$398570.03|
|  15| $404673.3|
|  16|$399586.53|
|  17|$398474.49|
|  18|$399332.91|
|  19|$398953.17|
+----+----------+
only showing top 20 rows

--- 0.6163218021392822 seconds ---


In [129]:

# 10. Partition by the "date_built" field on the formatted parquet home sales data

home_sales_revised.write.partitionBy("date_built").parquet("p_home_sales")



In [130]:
# 11. Read the parquet formatted data.

h_sales = spark.read.parquet("p_home_sales")


In [131]:
# 12. Create a temporary table for the parquet data.
h_sales.createOrReplaceTempView("p_temp_home_sales")



In [132]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""SELECT ROUND(AVG(price), 2) AS avg_price, view FROM p_temp_home_sales WHERE price >= 350000 group by view""").show()



print("--- %s seconds ---" % (time.time() - start_time))

+----------+----+
| avg_price|view|
+----------+----+
| 403005.77|   7|
| 788128.21|  51|
|  404673.3|  15|
| 798684.82|  54|
| 399548.12|  11|
| 397771.65|  29|
| 750537.94|  69|
|  396964.5|  42|
| 752861.18|  73|
| 1072285.2|  87|
| 767036.67|  64|
|  398867.6|   3|
|  397862.0|  30|
| 401419.75|  34|
|  791453.0|  59|
| 398592.71|   8|
| 402124.62|  28|
| 402022.68|  22|
|1056336.74|  85|
| 401934.21|  35|
+----------+----+
only showing top 20 rows

--- 1.9289898872375488 seconds ---


In [140]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")


DataFrame[]

In [142]:
from re import S
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached("home_sales")

False

# New Section